Spinorama plot with Altair

In [ ]:
import math
import pandas as pd
import altair as alt
import numpy as np
from src.spinorama.analysis import sound_power
from src.spinorama.load import parse_all_speakers, parse_graphs_speaker, graph_melt
from src.spinorama.graph import graph_params_default, graph_freq

# df = parse_graphs_speaker('Adam', 'Adam S2V', 'klippel')
# df = parse_graphs_speaker('Neumann', 'Neumann KH 80', 'klippel')
df = parse_graphs_speaker('Genelec', 'Genelec 8341A', 'klippel')
# df = parse_graphs_speaker('Genelec', 'Genelec 8030A', 'princeton')
# print(df)

In [ ]:
nearest = alt.selection(
    type='single',
    nearest=True,
    on='mouseover',
    fields=['Freq'],
    empty='none')

In [ ]:
def graph_spinorama(dfu, graph_params):
    xmin = graph_params['xmin']
    xmax = graph_params['xmax']
    ymin = graph_params['ymin']
    ymax = graph_params['ymax']
    if xmax == xmin:
        logging.error('Graph configuration is incorrect: xmin==xmax')
    if ymax == ymin:
        logging.error('Graph configuration is incorrect: ymin==ymax')
    # add selectors                                                                                                                                      
    selectorsMeasurements = alt.selection_multi(
        fields=['Measurements'],
        bind='legend')
    scales = alt.selection_interval(
        bind='scales'
    )
    # main charts                                                                                                                                        
    xaxis =alt.X('Freq:Q', title='Freqency (Hz)',
                scale=alt.Scale(type='log', base=10, nice=False, domain=[xmin, xmax]),
                axis=alt.Axis(format='s'))
    yaxis = alt.Y('dB:Q', scale=alt.Scale(zero=False, domain=[ymin, ymax]))
    di_yaxis = alt.Y('dB:Q', scale=alt.Scale(zero=False, domain=[0,ymax-ymin]))
    color = alt.Color('Measurements', type='nominal', sort=None)
    opacity = alt.condition(selectorsMeasurements, alt.value(1), alt.value(0.2))

    line=alt.Chart(dfu).mark_line().transform_filter(
        alt.FieldOneOfPredicate(
            field='Measurements',
            oneOf=['On Axis', 'Listening Window', 'Early Reflections', 'Sound Power'])
    ).encode(x=xaxis, y=yaxis, color=color, opacity=opacity
    )

    circle=alt.Chart(dfu).mark_circle(size=100).transform_filter(
        alt.FieldOneOfPredicate(
            field='Measurements',
            oneOf=['On Axis', 'Listening Window', 'Early Reflections', 'Sound Power'])
    ).encode(
        x=xaxis, y=yaxis, color=color,
        opacity=alt.condition(nearest, alt.value(1), alt.value(0)),
        tooltip=['Measurements', 'Freq', 'dB']
    ) #.transform_calculate(Freq=f'format(datum.Freq, ".0f")', dB=f'format(datum.dB, ".1f")')                                                            

    di=alt.Chart(dfu).mark_line().transform_filter(
        alt.FieldOneOfPredicate(
            field='Measurements',
            oneOf=['Early Reflections DI', 'Sound Power DI'])
    ).encode(x=xaxis, y=di_yaxis, color=color, opacity=opacity)

    circle_di = alt.Chart(dfu).mark_circle(size=100).transform_filter(
        alt.FieldOneOfPredicate(
            field='Measurements',
            oneOf=['Early Reflections DI', 'Sound Power DI'])
    ).encode(
        x=xaxis, y=di_yaxis, color=color,
        opacity=alt.condition(nearest, alt.value(1), alt.value(0)),
        tooltip=['Measurements', 'Freq', 'dB']
    ) #.transform_calculate(Freq=f'format(datum.Freq, ".0f")', dB=f'format(datum.dB, ".1f")')                                                            


    # assemble elements together                                                                                                                         
    spin = alt.layer(circle+line, circle_di+di).resolve_scale(y='independent'
    ).add_selection(
        selectorsMeasurements
    ).add_selection(
        scales
    ).add_selection(
        nearest
    ).properties(
        width=graph_params['width'],
        height=graph_params['height']
    ).interactive()
    return spin



dfu = df['CEA2034']
params = graph_params_default
params['ymin'] = -40
params['ymax'] = 10
params['width'] = 800
params['height'] = 500
graph_spinorama(dfu, params)

Compare computed sound power wrt the version done by Klippel
Matching is good for low frequency but error increases with frequency

In [ ]:
#df = pd.DataFrame({'Freq': [200, 2000, 10000, 12000, 16000, 20000], '10°': [2,2,2,2,2,2], '-10°': [2,2,2,2,2,2]})
sp = sound_power(df['SPL Horizontal_unmelted'], df['SPL Vertical_unmelted'])

In [ ]:
dfu = df['CEA2034_unmelted']
check = pd.DataFrame({
    'Freq': dfu.Freq, 
    'Control': dfu['Sound Power'], 
    'Computed': sp.dB})
mcheck = graph_melt(check)
alt.Chart(mcheck).mark_line(clip=True).encode(
    x=alt.X('Freq', scale=alt.Scale(type='log', nice=False)),
    y=alt.Y('dB', scale=alt.Scale(domain=[50, 96])),
    color=alt.Color('Measurements'))

In [ ]:
onaxis = df['CEA2034']
onaxis = onaxis.loc[onaxis['Measurements'] == 'On Axis']
onaxis_graph = graph_freq(onaxis, graph_params_default)
onaxis_reg = alt.Chart(onaxis).transform_filter(
   'datum.Freq>80 & datum.Freq<10000',
).transform_regression(method='log', on='Freq', regression='dB', extent=[20,20000]
).mark_line().encode(
   alt.X('Freq:Q'),
   alt.Y('dB:Q'),
   color=alt.value('red')
)
onaxis_graph + onaxis_reg


In [ ]:
inroom = df['Estimated In-Room Response']
inroom_graph = graph_freq(inroom, graph_params_default)
inroom_reg = alt.Chart(inroom
).transform_filter('datum.Freq>80 & datum.Freq<10000'
).transform_regression(method='log', on='Freq', regression='dB', extent=[20,20000]
).mark_line(
).encode(alt.X('Freq:Q'), alt.Y('dB:Q'), color=alt.value('red'))
inroom_graph + inroom_reg